In [0]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [0]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [0]:
x=df.iloc[:,1:]

In [0]:
y=df['total_bill']

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [0]:
numeric_preprocessor = Pipeline(
 steps=[
 ("imputation_mean", SimpleImputer(missing_values=np.nan,
strategy="mean")),
 ("scaler", StandardScaler()),
 ]
)


In [0]:
from sklearn import set_config
set_config(display='diagram')
numeric_preprocessor

Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('scaler', StandardScaler())])

In [0]:
categorical_preprocessor = Pipeline(
 steps=[
 (
 "imputation_constant",
 SimpleImputer(fill_value="missing", strategy="constant"),
 ),
 ("onehot", OneHotEncoder(handle_unknown="ignore")),
 ]
)


In [0]:
preprocessor=Pipeline(
 steps=[("categorical",categorical_preprocessor),
("numerical",numeric_preprocessor)]
)

In [0]:
preprocessor

Pipeline(steps=[('categorical',
                 Pipeline(steps=[('imputation_constant',
                                  SimpleImputer(fill_value='missing',
                                                strategy='constant')),
                                 ('onehot',
                                  OneHotEncoder(handle_unknown='ignore'))])),
                ('numerical',
                 Pipeline(steps=[('imputation_mean', SimpleImputer()),
                                 ('scaler', StandardScaler())]))])

In [0]:
pipe=Pipeline(
 [("preprocessor",preprocessor),
("regressor",RandomForestRegressor())]
)


In [0]:
pipe

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('categorical',
                                  Pipeline(steps=[('imputation_constant',
                                                   SimpleImputer(fill_value='missing',
                                                                 strategy='constant')),
                                                  ('onehot',
                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                 ('numerical',
                                  Pipeline(steps=[('imputation_mean',
                                                   SimpleImputer()),
                                                  ('scaler',
                                                   StandardScaler())]))])),
                ('regressor', RandomForestRegressor())])

In [0]:
from sklearn.compose import ColumnTransformer

categorical_cols = x.select_dtypes(include='object').columns.tolist()
numerical_cols = x.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", categorical_preprocessor, categorical_cols),
        ("numerical", numeric_preprocessor, numerical_cols),
    ]
)

pipe = Pipeline(
    [("preprocessor", preprocessor), ("regressor", RandomForestRegressor())]
)

pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  []),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor', RandomForestRegressor())])